In [1]:
#IMPORTS
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import networkx as nx
from tkinter import *
from tkinter import ttk
import json
#from collections import deque
from matplotlib import lines
import random

import structures
import uninformedsearches as uns

In [2]:
# dictionary to hold user input
user_data = {}

In [21]:
def retrieve_entries():
    
    user_data.clear()

    i=0
    
    for entry in vertex_list:
        
        # 0 value for nested keys as a placeholder for informed search expansion 
        user_data[entry.get()] = {edge_list[i].get():0, edge_list[i+1].get():0, edge_list[i+2].get():0, edge_list[i+3].get():0}
        
        i+=4
        
    update_cbox()

        
def save_data():

    try:
        retrieve_entries()
    
        with open("backup.json","w") as file:
            json.dump(user_data,file)
    
                
        set_feedback("success", "Success: Data saved to backup.json")

    except: 
        set_feedback("error", "Error: Check entered data is correct")
        
def load_data():
    global user_data
    try:
        with open("backup.json") as read_file:
            user_data = json.load(read_file)

        main_keys = list(user_data.keys())
    
        clear_input()
    
        i = 0
        for entry in main_keys:
            vertex_list[i].insert(0, entry)
            i+=1

        
        j=0
        for entry in main_keys:
        
            nested_keys = list(user_data[entry].keys())
        
            # simplifies iterating over four entry boxs per line
            while len(nested_keys) < 4:
                nested_keys.append("")
        
            for entry in nested_keys:
                edge_list[j].insert(0, entry)
                j+=1
            
        update_cbox()
            
        set_feedback("success", "Success: Data loaded from backup.json")

    except: 
        set_feedback("error", "Error: Check backup.json exists in this directory")
            
def clear_input():
    
    for entry in vertex_list:
        entry.delete(0, END)
        
    for entry in edge_list:
        entry.delete(0, END)
        
def set_feedback(status, message):

    if status == "error":
        feedback_label.config(text=message, background="#FF3546")  # red error bg
    elif status == "success":
        feedback_label.config(text=message, background="#51D12E")  # green error bg

    feedback_label.after(2500, lambda: feedback_label.config(text="", background="#DFDFDF"))
    
def update_cbox():

    # filters any blank keys for aesthetic drop down lists 
    tidied_list = list(filter(None, list(user_data.keys())))
    
    start_cbox["values"]  = tidied_list
    end_cbox["values"] = tidied_list


def combo_retrieve(cbox):

    value = cbox.get()

    return value



def draw_graph(graph, node_colors="white"):

    graph_plot.cla()

    G = nx.Graph(graph.data)
    pos=nx.spring_layout(G, seed=graph_seed)
    
    nx.draw(
        G,
        pos=pos,
        with_labels=False,
        ax=graph_plot,
        node_color=node_colors,
        edge_color="dimgray",
        edgecolors="dimgray",
        node_size=500,
        width=1,
    )

    graph_figure.set_facecolor("#f0f0f0")
    
    
    
    def nudge(pos, x_shift, y_shift):
        return {n:(x + x_shift, y + y_shift) for n,(x,y) in pos.items()}
   
    pos_nodes = nudge(pos, 0, -.079) 
    
    node_label_handles = nx.draw_networkx_labels(G, pos=pos_nodes,ax=graph_plot,font_size=11)    
    
    # add a white bounding box behind the node labels
    [label.set_bbox(dict(facecolor='#f0f0f0', edgecolor='none')) for label in node_label_handles.values()]

    # add a legend
    white_circle = lines.Line2D([], [], color="dimgray", marker='o', markersize=16, markerfacecolor="white")
    green_circle = lines.Line2D([], [], color="dimgray", marker='o', markersize=16, markerfacecolor="green")
    graph_plot.legend((white_circle ,green_circle),
               ('Node', 'Solution'),
               numpoints=1, prop={'size': 11}, loc=(.9175, .9175))
    
    graph_canvas.draw()
  
graph_seed = 10

def redraw_graph():
    
    global graph_seed
    
    graph_seed = random.randint(1,100)
    
    create_graph()
    
    
    
    


def create_graph():
    global user_graph
    
    user_graph = structures.Graph(user_data, combo_retrieve(graph_cbox))
    draw_graph(user_graph)    
    
    
    
def create_problem():

    user_problem = structures.Problem(combo_retrieve(start_cbox), combo_retrieve(end_cbox), user_graph)
    
    return user_problem


def search_problem(search):
    
    user_problem = create_problem()
      
    search_result = uns.breadth_first_graph_search(user_problem)

    if search_result is not None:
    
        search_result = search_result.solution()
        search_result.insert(0, user_problem.initial)

        node_colors = ["green" if node in search_result else "white" for node in user_graph.nodes()]
        draw_graph(user_graph, node_colors)
        set_feedback("success", "Success: Solution path displayed in green")
    else:
        set_feedback("error", "Error: No path found")  

In [22]:
# ROOT

root = Tk()
root.title("Uninformed Search Tool")
root.resizable(False, False)


# FRAMES

## frames used to provide underlying structure for the gui

feedback_frame = Frame(root, bd=1, bg="#a0a0a0")
feedback_frame.pack(padx=10, pady=10, side="bottom", fill="x")

data_lframe = LabelFrame(root, text="Data")
data_lframe.pack(padx=(10,5), pady=(10,0), side="left", fill="y")

search_lframe = LabelFrame(root, text="Search")
search_lframe.pack(padx=(5,10), pady=(10,0), side="right", fill="y")
   

# BUTTONS

## data label frame

draw_button = Button(data_lframe, text="Draw", width=27, height=1, command=create_graph, relief=GROOVE)
draw_button.grid(row=22, column=0, columnspan=3, padx=(12,3), pady=(8,11))

save_button = Button(data_lframe,text="Save",width=7,height=1,command=save_data,relief=GROOVE)
save_button.grid(row=22, column=3, pady=(8,11))

load_button = Button(data_lframe, text="Load", width=7, height=1, command=load_data, relief=GROOVE)
load_button.grid(row=22, column=4, pady=(8,11))

## search label frame

start_button = Button(search_lframe, text="Start Search", width=147, height=1, command=lambda: search_problem(combo_retrieve(algorithm_cbox)) , relief=GROOVE)
start_button.grid(row=2, column=0, columnspan=6)

redraw = Button(search_lframe, text="Redraw", width=7, height=1, command=redraw_graph, relief=GROOVE)
redraw.grid(row=4, column=5, padx=12, pady=(0,11), sticky="e")

# ENTRY - fields for user input

## loops are used to create and place the 100 needed entry fields

vertex_list = []

for i in range(20):
    vertex_list.append(Entry(data_lframe, width=9))

vertex_row = 2 # starts placing fields on row 2
for entry in vertex_list:
    entry.grid(row=vertex_row, column=0, padx=(12,8), pady=(4,5) ,sticky="w")
    vertex_row += 1

edge_list = []

## loop to create 60 edge entry widget
for i in range(80):
    edge_list.append(Entry(data_lframe, width=9))

## loop to create and place all edge entry widgets

edge_row = 2 # starts placing fields on row 2
edge_column = 1 # starts placing fields in column 1
for entry in edge_list:
    entry.grid(row=edge_row, column=edge_column, padx=0, sticky="w")
    if edge_column == 4: # the maximum desired edge fields per row
        edge_column = 1
        edge_row += 1
    else:
        edge_column += 1


# LABELS

## data label frame

graph_label = Label(data_lframe, text="Graph Type:")
graph_label.grid(row=0, column=0, padx=10, pady=(12,2), sticky="w")

vertex_label = Label(data_lframe, text="Vertices:")
vertex_label.grid(row=1, column=0, padx=10, pady=12, sticky="w")

edge_label = Label(data_lframe, text="Edges:")
edge_label.grid(row=1, column=1, padx=0, pady=12, sticky="w")

## search label frame

search_label = Label(search_lframe, text="Search:")
search_label.grid(row=0, column=0, padx=10, pady=12, sticky="w")

start_label = Label(search_lframe, text="Start:")
start_label.grid(row=0, column=2, padx=(75,10), pady=12, sticky="w")

end_label = Label(search_lframe, text="Goal:")
end_label.grid(row=0, column=4, padx=(75,10), pady=12, sticky="w")

## feedback frame

feedback_label = Label(feedback_frame, text="", background="#DFDFDF")
feedback_label.pack(fill="x")


# COMBO BOXES

## data label frame

graph_list = ["Undirected (will correct for asymmetry) ", "Directed"]
graph_choice = StringVar(value=graph_list[0])
graph_cbox = ttk.Combobox(data_lframe, textvariable=graph_choice, width=35)
graph_cbox["values"] = graph_list
graph_cbox.grid(row=0, column=1, columnspan=6, padx=(10,14), pady=(12,2))

## search label frame

algorithm_list = ["Breadth First Search","Depth First Search","Iterative Deepening Depth First Search"]
algorithm_choice = StringVar(value=algorithm_list[0])
algorithm_cbox = ttk.Combobox(search_lframe, textvariable=algorithm_choice, width=35)
algorithm_cbox["values"] = algorithm_list
algorithm_cbox.grid(row=0, column=1, padx=10, pady=12)

start_choice = StringVar()
start_cbox = ttk.Combobox(search_lframe, textvariable=start_choice, width=35)
start_cbox["values"] = ()
start_cbox.grid(row=0, column=3, padx=10, pady=12)

end_choice = StringVar()
end_cbox = ttk.Combobox(search_lframe, textvariable=end_choice, width=35)
end_cbox["values"] = ()
end_cbox.grid(row=0, column=5, padx=(10,14), pady=12)


# FIGURES

graph_figure = Figure(figsize=(14.4, 7.88))
graph_figure.set_facecolor("#f0f0f0")



# SUBPLOTS

graph_plot = graph_figure.add_subplot(111)
graph_plot.axis("off")
graph_figure.tight_layout(pad=0, h_pad=0, w_pad=0, rect=None)

# CANVAS

graph_canvas = FigureCanvasTkAgg(graph_figure, search_lframe)
graph_canvas.get_tk_widget().grid(row=3, column=0, columnspan=6, padx=0, pady=(10,0))


# LOOP

root.mainloop()